# Step 4: Test and Compare Models

In this step, you'll test your trained model and compare it with EasyOCR's pre-trained model to evaluate performance.

## What You'll Learn:
- How to set up a custom OCR model for inference
- How to compare custom vs pre-trained models
- How to interpret OCR results and confidence scores

## 4.1 Setup: Copy Trained Model

First, copy your trained model to the `user_network_dir` directory where EasyOCR will load it.

**Important Files Needed:**
- `user_network_dir/custom.pth` - Your trained model weights
- `user_network_dir/custom.yaml` - Model configuration (architecture, character list)
- `user_network_dir/custom.py` - Model architecture definition

In [6]:
# Create the user network directory if it doesn't exist
#!mkdir -p ./user_network_dir

# Copy your trained model to the user network directory
# ⚠️ IMPORTANT: Change the date folder name to match your training run
# The folder name format is: TPS-VGG-BiLSTM-CTC-Seed1111-YYYYMMDD
#!cp ./saved_models/TPS-VGG-BiLSTM-CTC-Seed1111/best_accuracy.pth ./user_network_dir/custom.pth
# %pip install easyocr
# %pip install torch opencv-python-headless python-bidi
#%pip install --upgrade easyocr torch torchvision torchaudio
%pip show easyocr

Name: easyocr
Version: 1.7.2
Summary: End-to-End Multi-Lingual Optical Character Recognition (OCR) Solution
Home-page: https://github.com/jaidedai/easyocr
Author: Rakpong Kittinaradorn
Author-email: r.kittinaradorn@gmail.com
License: Apache License 2.0
Location: c:\Users\user\anaconda3\Lib\site-packages
Requires: ninja, numpy, opencv-python-headless, Pillow, pyclipper, python-bidi, PyYAML, scikit-image, scipy, Shapely, torch, torchvision
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## 4.2 Prepare Test Images

Place your test images in the `demo_images` folder. These should be images similar to what your model will process in production (e.g., receipts, invoices, signs).

```bash
demo_images/
  ├── demo_image_01.jpg
  ├── demo_image_02.jpg
  └── ...
```

## 4.3 Test with Pre-trained Model (Baseline)

First, test with EasyOCR's pre-trained model to establish a baseline performance.

In [1]:
from easyocr import Reader
import os

# Set GPU devices
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# Initialize EasyOCR with pre-trained traditional Chinese model
print("Loading EasyOCR pre-trained model...")
reader_pretrained = Reader(['ch_tra'], gpu=True)

# Get test images
demo_images_dir = './demo_images'
test_files = [f for f in os.listdir(demo_images_dir) if not f.startswith('.')]
test_files.sort()

print(f"\n{'='*70}")
print("TESTING WITH EASYOCR PRE-TRAINED MODEL (Baseline)")
print(f"{'='*70}\n")

# Test each image - simplified version
for filename in test_files:
    file_path = os.path.join(demo_images_dir, filename)
    results = reader_pretrained.readtext(file_path)
    
    for bbox, text, confidence in results:
        print("filename: '%s', confidence: %.4f, string: '%s'" % (filename, confidence, text))

Loading EasyOCR pre-trained model...

TESTING WITH EASYOCR PRE-TRAINED MODEL (Baseline)

filename: 'demo_image_01.jpg', confidence: 0.5220, string: '九龍青衣細山路2號美景花園平臺72號舖'


## 4.4 Test with Your Custom Model

Now test with your trained model. Make sure you have:
- ✅ `user_network_dir/custom.pth` (model weights)
- ✅ `user_network_dir/custom.yaml` (configuration)
- ✅ `user_network_dir/custom.py` (architecture)

In [3]:
from easyocr import Reader
import os

# Set GPU devices
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# Initialize EasyOCR with your custom model
print("Loading your custom trained model...")
reader_custom = Reader(
    ['ch_tra'], 
    gpu=True,
    model_storage_directory='./user_network_dir',
    user_network_directory='./user_network_dir',
    recog_network='custom'
)

# Get test images
demo_images_dir = './demo_images'
test_files = [f for f in os.listdir(demo_images_dir) if not f.startswith('.')]
test_files.sort()

print(f"\n{'='*70}")
print("TESTING WITH YOUR CUSTOM MODEL")
print(f"{'='*70}\n")

# Test each image - simplified version
for filename in test_files:
    file_path = os.path.join(demo_images_dir, filename)
    results = reader_custom.readtext(file_path)
    
    for bbox, text, confidence in results:
        print("filename: '%s', confidence: %.4f, string: '%s'" % (filename, confidence, text))

Loading your custom trained model...


RuntimeError: Error(s) in loading state_dict for DataParallel:
	Missing key(s) in state_dict: "module.FeatureExtraction.ConvNet.0.weight", "module.FeatureExtraction.ConvNet.0.bias", "module.FeatureExtraction.ConvNet.3.weight", "module.FeatureExtraction.ConvNet.3.bias", "module.FeatureExtraction.ConvNet.6.weight", "module.FeatureExtraction.ConvNet.6.bias", "module.FeatureExtraction.ConvNet.8.weight", "module.FeatureExtraction.ConvNet.8.bias", "module.FeatureExtraction.ConvNet.11.weight", "module.FeatureExtraction.ConvNet.12.weight", "module.FeatureExtraction.ConvNet.12.bias", "module.FeatureExtraction.ConvNet.12.running_mean", "module.FeatureExtraction.ConvNet.12.running_var", "module.FeatureExtraction.ConvNet.14.weight", "module.FeatureExtraction.ConvNet.15.weight", "module.FeatureExtraction.ConvNet.15.bias", "module.FeatureExtraction.ConvNet.15.running_mean", "module.FeatureExtraction.ConvNet.15.running_var", "module.FeatureExtraction.ConvNet.18.weight", "module.FeatureExtraction.Conv

## 4.5 Side-by-Side Comparison

Compare both models on the same images to see which performs better.

In [3]:
from easyocr import Reader
import os

# Set GPU devices
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# Initialize both models
print("Loading both models...")
reader_pretrained = Reader(['ch_tra'], gpu=True)
reader_custom = Reader(['ch_tra'], gpu=True,
                      model_storage_directory='./user_network_dir',
                      user_network_directory='./user_network_dir',
                      recog_network='custom')

# Get test images
demo_images_dir = './demo_images'
test_files = [f for f in os.listdir(demo_images_dir) if not f.startswith('.')]
test_files.sort()

print(f"\n{'='*80}")
print("SIDE-BY-SIDE COMPARISON: Pre-trained vs Custom Model")
print(f"{'='*80}\n")

# Compare results for each image
for filename in test_files:
    file_path = os.path.join(demo_images_dir, filename)
    print(f"📷 Testing: {filename}")
    print(f"{'-'*80}")
    
    # Test with pre-trained model
    results_pretrained = reader_pretrained.readtext(file_path)
    print("🔵 EasyOCR Pre-trained Model:")
    if results_pretrained:
        for bbox, text, confidence in results_pretrained:
            print("   confidence: %.4f, string: '%s'" % (confidence, text))
    else:
        print("   No text detected")
    
    print()
    
    # Test with custom model
    results_custom = reader_custom.readtext(file_path)
    print("🟢 Your Custom Model:")
    if results_custom:
        for bbox, text, confidence in results_custom:
            print("   confidence: %.4f, string: '%s'" % (confidence, text))
    else:
        print("   No text detected")
    
    print(f"\n{'='*80}\n")

Loading both models...

SIDE-BY-SIDE COMPARISON: Pre-trained vs Custom Model

📷 Testing: demo_image_01.jpg
--------------------------------------------------------------------------------
🔵 EasyOCR Pre-trained Model:
   confidence: 0.0148, string: '九龍青衣紐山路2號美景花園平壼72號舖'

🟢 Your Custom Model:
   confidence: 0.0000, string: ''




In [11]:
from easyocr import Reader
import os

# Set GPU devices
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# Initialize EasyOCR with your custom model
# It will automatically find and use custom.yaml in the user_network_directory
print("Loading your custom trained model...")
reader_custom = Reader(
    ['ch_tra'],
    gpu=True,
    model_storage_directory='./user_network_dir',
    user_network_directory='./user_network_dir', # This directory MUST contain custom.pth and custom.yaml
    recog_network='custom'
)

# --- The rest of your testing code remains the same ---

# Get test images
demo_images_dir = './demo_images'
test_files = [f for f in os.listdir(demo_images_dir) if not f.startswith('.')]
test_files.sort()

print(f"\n{'='*70}")
print("TESTING WITH YOUR CUSTOM MODEL")
print(f"{'='*70}\n")

# Test each image
for filename in test_files:
    file_path = os.path.join(demo_images_dir, filename)
    results = reader_custom.readtext(file_path)

    for bbox, text, confidence in results:
        print("filename: '%s', confidence: %.4f, string: '%s'" % (filename, confidence, text))

Loading your custom trained model...


LinAlgError: Last 2 dimensions of the array must be square